# Using IBM Cloud SQL Query

<div class="pull-left"><left><img style="float: right;" src="http://developer.ibm.com/clouddataservices/wp-content/uploads/sites/85/2018/01/ibm-cloud-object-storage-logo-small.png" width="100" margin=50></left></div>
<div style="text-align:center">
IBM Cloud SQL Query is IBM's serverless SQL service on data in Cloud Object Storage. It allows to run ANSI SQL on Parquet, CSV, JSON, ORC and AVRO data sets. You can use it to run your analytic queries, and you can use it to conduct complex transformations and write the result in any desired data format, partitioning and layout. SQL Query is based on Apache Spark SQL as the query engine in the background. This means you do not have to provision any Apache Spark instance or service. A simple Python client (like the IBM Watson Studio Notebook) is sufficient.<br><br></div>
This notebook is meant to be a generic starter to use the SQL Query API in order to run SQL statements in a programmatic way. It uses the <a href="https://github.com/IBM-Cloud/sql-query-clients/tree/master/Python" target="_blank" rel="noopener noreferrer">ibmcloudsql</a> Python library for this purpose. The notebook also demonstrates how you can combine SQL Query with visualization libraries such as PixieDust. The notebook has been verified to work with Python 3.5. As mentioned above it does not require a Spark service bound to the notebook.

## Table of contents
1. [Setup libraries](#setup)<br>
2. [Configure SQL Query](#configure)<br>
    2.1 [Using the project bucket](#projectbucket)<br>
    2.2 [Setting SQL Query parameters](#parameters)<br>
3. [Your SQL](#sql)<br>
4. [Running Your SQL Statement](#run)<br>
    4.1 [Low level SQL job submission](#lowlevel)<br>
5. [Running ETL SQLs](#etl)<br>
6. [Paginated SQL Results](#pagination)<br>
7. [Working with SQL result objects](#results)<br>
8. [List recent SQL submissions](#joblist)<br>
9. [Next steps](#next)<br>

### <a id="setup"></a> 1. Setup libraries

Run the following cell at least once in your notebook environment in order to install required packages, such as the SQL Query client library:

In [ ]:
!conda install pyarrow
!conda install sqlparse

In [ ]:
!pip install --upgrade --user ibmcloudsql

In [ ]:
import ibmcloudsql
from pixiedust.display import *
import pandas as pd
targeturl=''

### <a id="configure"></a> 2. Configure SQL Query
1. You need an **API key** for an IBM cloud identity that has access to your Cloud Object Storage bucket for writing SQL results and to your SQL Query instance. To create API keys log on to the IBM Cloud console and go to <a href="https://console.bluemix.net/iam/#/apikeys" target="_blank">Manage->Security->Platform API Keys</a>, click the `Create` button, give the key a custom name and click `Create`. In the next dialog click `Show` and copy the key to your clipboard and paste it below in this notebook.
2. You need the **instance CRN** for the SQL Query instance. You can find it in the <a href="https://console.bluemix.net/dashboard/apps" target="_blank">IBM Cloud console dashboard</a>. Make sure you have `All Resources` selected as resource group. In the section `Services` you can see your instances of SQL Query and Cloud Object Storage. Select the instance of SQL Query that you want to use. In the SQL Query dashboard page that opens up you find a section titled **REST API** with a button labelled **Instance CRN**. Click the button to copy the CRN into your clipboard and paste it here into the notebook. If you don't have an SQL Query instance created yet, <a href="https://console.bluemix.net/catalog/services/sql-query" target="_blank">create one</a> first.
3. You need to specify the location on Cloud Object Storage where your **query results** should be written. This comprises three parts of information that you can find in the Cloud Object Storage UI for your instance in the IBM Cloud console. You need to provide it as a **URL** using the format `cos://<endpoint>/<bucket>/[<prefix>]`. You have the option to use the cloud object storage **bucket that is associated with your project**. In this case, execute the following section before you proceed.  
<br/>
For more background information, check out the SQL Query <a href="https://console.bluemix.net/docs/services/sql-query/getting-started.html#getting-started-tutorial" target="_blank">documentation</a>.

#### <a id="projectbucket"></a> 2.1 Using the project bucket
**Only** follow the instructions in this section when you want to write your SQL query results to the bucket that has been created for the project for which you have created this notebook. In any other case proceed directly with section **2.2**.
<br><br>
__Inserting the project token__:  
Click the `More` option in the toolbar above (the three stacked dots) and select `Insert project token`.
 * If you haven't created an access token for this project before, you will see a dialog that asks you to create one first. Follow the link to open your project settings, scroll down to `Access tokens` and click `New token`. Give the token a custom name and make sure you select `Editor` as `Access role for project`. After you created your access token you can come back to this notebook, select the empty cell below and again select `Insert project token` from the toolbar at the top.
[//]: # 
This will add a new cell at the top of your notebook with content that looks like this:
```
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='<some id>', project_access_token='<some access token>')
pc = project.project_context
```
Leave that cell content as inserted and run the cell. Then then proceed with the following cell below:

In [ ]:
cos_bucket = project.get_metadata()['entity']['storage']['properties']
targeturl="cos://" + cos_bucket['bucket_region'] + "/" + cos_bucket['bucket_name'] + "/"

#### <a id="parameters"></a> 2.2 Setting the SQL Query parameters

In [5]:
import getpass
apikey=getpass.getpass('Enter IBM Cloud API Key (leave empty to use previous one): ') or apikey
instnacecrn=input('Enter SQL Query Instance CRN (leave empty to use previous one): ') or instnacecrn
if targeturl == '':
    targeturl=input('Enter target URL for SQL results: ')
else:
    targeturl=input('Enter target URL for SQL results (leave empty to use ' + targeturl + '): ') or targeturl
sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, client_info='SQL Query Starter Notebook')
sqlClient.logon()
print('\nYour SQL Query web console link:\n')
sqlClient.sql_ui_link()

Enter IBM Cloud API Key (leave empty to use previous one): ········
Enter SQL Query Instance CRN (leave empty to use previous one): crn%3Av1%3Abluemix%3Apublic%3Asql-query%3Aus-south%3Aa%2Fd86af7367f70fba4f306d3c19c938f2f%3Ad1b2c005-e3d8-48c0-9247-e9726a7ed510%3A%3A
Enter target URL for SQL results: cos://us-south/sqltempregional/

Your SQL Query web console link:

https://sql-query.cloud.ibm.com/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/d86af7367f70fba4f306d3c19c938f2f:d1b2c005-e3d8-48c0-9247-e9726a7ed510::


### <a id="sql"></a> 3. Your SQL
To author your own SQL query, use the interactive SQL Query web console (**link above**) of your SQL Query service instance.

In [6]:
import sqlparse
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter

sql=input('Enter your SQL statement (leave empty to use a simple sample SQL)')
if sql == '':
    sql='SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, \
         cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c \
         WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01" \
         ORDER BY c.CompanyName'
if " INTO " not in sql:
    sql += ' INTO {} STORED AS CSV'.format(targeturl)
formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='tango')
result = highlight(formatted_sql, lexer, formatter)
from IPython.core.display import display, HTML
print('\nYour SQL statement is:\n')
print(result)

Enter your SQL statement (leave empty to use a simple sample SQL)

Your SQL statement is:

SELECT o.OrderID,
	c.CompanyName,
	e.FirstName,
	e.LastName
FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o,          cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c          WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01"          ORDER BY c.CompanyName INTO cos://us-south/sqltempregional/ STORED AS CSV



### <a id="run"></a> 4. Running Your SQL Statement
The following cell submits the above SQL statement and waits for it to finish before printing a sample of the result set.

In [7]:
result_df = sqlClient.run_sql(sql)
if isinstance(result_df, str):
    print(result_df)

In [8]:
result_df.head(10)

,OrderID,CompanyName,FirstName,LastName
0,10924,Berglunds snabbköp,Janet,Leverling
1,11058,Blauer See Delikatessen,Anne,Dodsworth
2,10827,Bon app',Nancy,Davolio
3,11076,Bon app',Margaret,Peacock
4,11045,Bottom-Dollar Markets,Michael,Suyama
5,10970,Bólido Comidas preparadas,Anne,Dodsworth
6,11054,Cactus Comidas para llevar,Laura,Callahan
7,11008,Ernst Handel,Robert,King
8,11072,Ernst Handel,Margaret,Peacock
9,10816,Great Lakes Food Market,Margaret,Peacock


In [9]:
from pixiedust.display import *
display(result_df)

#### <a id="lowlevel"></a> 4.1 Low level SQL job submission
Let's run the same SQL again, but this time using the asynchronous submission mechanism and the status check method.

In [10]:
sqlClient.logon()
jobId = sqlClient.submit_sql(sql)
print("SQL query submitted and running in the background. jobId = " + jobId)

SQL query submitted and running in the background. jobId = 2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855


In [11]:
print("Job status for " + jobId + ": " + sqlClient.get_job(jobId)['status'])

Job status for 2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855: running


Use the `wait_for_job()` method as a blocking call until your job has finished:

In [12]:
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
if job_status == 'failed':
    details = sqlClient.get_job(jobId)
    print("Error: {}\nError Message: {}".format(details['error'], details['error_message']))

Job 2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855 terminated with status: completed


Use the `get_result()` method to retrieve a dataframe for the SQL result set:

In [13]:
result_df = sqlClient.get_result(jobId)
print("OK, we have a dataframe for the SQL result that has been stored by SQL Query in " + sqlClient.get_job(jobId)['resultset_location'])

OK, we have a dataframe for the SQL result that has been stored by SQL Query in cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/jobid=2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855


You can delete the result set from Cloud Object Storage using the `delete_result()` method:

In [14]:
sqlClient.delete_result(jobId)

,Deleted Object
0,jobid=2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855/_SU...
1,jobid=2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855/par...
2,jobid=2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855


### <a id="etl"></a> 5. Running ETL SQLs
The following ETL SQL statement joins two data sets from COS and writes the result to COS using **hive style partitioning** with two columns. 

In [15]:
etl_sql='SELECT OrderID, c.CustomerID CustomerID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax \
         EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, \
         ShipCity, ShipRegion, ShipPostalCode, ShipCountry FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, \
         cos://us-geo/sql/customers.parquet STORED AS PARQUET c \
         WHERE c.CustomerID = o.CustomerID \
         INTO {}customer_orders STORED AS PARQUET PARTITIONED BY (ShipCountry, ShipCity)'.format(targeturl)
formatted_etl_sql = sqlparse.format(etl_sql, reindent=True, indent_tabs=True, keyword_case='upper')
result = highlight(formatted_etl_sql, lexer, formatter)
print('\nExample ETL Statement is:\n')
print(result)


Example ETL Statement is:

SELECT OrderID,
	c.CustomerID CustomerID,
	CompanyName,
	ContactName,
	ContactTitle,
	Address,
	City,
	Region,
	PostalCode,
	Country,
	Phone,
	Fax EmployeeID,
	OrderDate,
	RequiredDate,
	ShippedDate,
	ShipVia,
	Freight,
	ShipName,
	ShipAddress,
	ShipCity,
	ShipRegion,
	ShipPostalCode,
	ShipCountry
FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o,          cos://us-geo/sql/customers.parquet STORED AS PARQUET c          WHERE c.CustomerID = o.CustomerID          INTO cos://us-south/sqltempregional/customer_orders STORED AS PARQUET PARTITIONED BY (ShipCountry, ShipCity)



In [16]:
jobId = sqlClient.submit_sql(etl_sql)
print("SQL query submitted and running in the background. jobId = " + jobId)
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
job_details = sqlClient.get_job(jobId)
if job_status == 'failed':
    print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))

SQL query submitted and running in the background. jobId = 16cb6bcc-56e6-4c84-9dd2-6d9ee344e954
Job 16cb6bcc-56e6-4c84-9dd2-6d9ee344e954 terminated with status: completed


The following cell uses the `get_cos_summary()` method to print a summary of the objects that have been written by the previous ETL SQL statement. Note the **total_volume** value. We will reference it for comparison in the next steps.

In [17]:
resultset_location = job_details['resultset_location']
sqlClient.get_cos_summary(resultset_location)

{'url': 'cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954',
 'total_objects': 72,
 'total_volume': '401.7 KB',
 'oldest_object_timestamp': 'February 13, 2020, 13H:45M:54S',
 'newest_object_timestamp': 'February 13, 2020, 13H:46M:17S',
 'smallest_object_size': '0.0 B',
 'smallest_object': 'customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954',
 'largest_object_size': '7.1 KB',
 'largest_object': 'customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=UK/ShipCity=London/part-00015-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134603_0006_m_000015_0.c000.snappy.parquet'}

The following cell uses the `list_results()` method to print a list of the objects that have been written by the above ETL SQL statement. Note the partition columns and their values being part of the object names now. This naming convention is known as **hive style partitioning**. This type of partitioning is the basis for optimizing SQL queries using predicates that match with the partitioning columns.

In [18]:
pd.set_option('display.max_colwidth', -1)
result_objects_df = sqlClient.list_results(jobId)
print("List of objects written by ETL SQL:")
result_objects_df.head(200)

List of objects written by ETL SQL:


,ObjectURL,Size,Bucket,Object
0,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954,0,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954
1,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Argentina/ShipCity=Buenos Aires/part-00065-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134609_0006_m_000065_0.c000.snappy.parquet,6412,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Argentina/ShipCity=Buenos Aires/part-00065-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134609_0006_m_000065_0.c000.snappy.parquet
2,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Austria/ShipCity=Graz/part-00096-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134610_0006_m_000096_0.c000.snappy.parquet,6094,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Austria/ShipCity=Graz/part-00096-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134610_0006_m_000096_0.c000.snappy.parquet
3,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Austria/ShipCity=Salzburg/part-00015-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134603_0006_m_000015_0.c000.snappy.parquet,5636,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Austria/ShipCity=Salzburg/part-00015-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134603_0006_m_000015_0.c000.snappy.parquet
4,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Belgium/ShipCity=Bruxelles/part-00180-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134609_0006_m_000180_0.c000.snappy.parquet,5660,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Belgium/ShipCity=Bruxelles/part-00180-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134609_0006_m_000180_0.c000.snappy.parquet
5,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Belgium/ShipCity=Charleroi/part-00006-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134601_0006_m_000006_0.c000.snappy.parquet,5957,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Belgium/ShipCity=Charleroi/part-00006-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134601_0006_m_000006_0.c000.snappy.parquet
6,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Brazil/ShipCity=Campinas/part-00069-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134609_0006_m_000069_0.c000.snappy.parquet,5715,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Brazil/ShipCity=Campinas/part-00069-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134609_0006_m_000069_0.c000.snappy.parquet
7,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Brazil/ShipCity=Resende/part-00042-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134605_0006_m_000042_0.c000.snappy.parquet,5749,sqltempregional,customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Brazil/ShipCity=Resende/part-00042-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134605_0006_m_000042_0.c000.snappy.parquet
8,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954/ShipCountry=Brazil/ShipCity=Rio de Janeiro/part-00198-ab4967aa-99a6-485a-8976-95b2af6d0869-attempt_20200213134610

Now let's take a look at the result data with the `get_result()` method. Note that the result dataframe contains the two partitioning columns. The values for these have been put together by get_result() from the object names above because in hive style partitioning the partition column values are not stored in the objects but rather in the object names.

In [19]:
sqlClient.get_result(jobId).head(100)

,OrderID,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,...,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipRegion,ShipPostalCode,ShipCountry,ShipCity
0,10409,OCEAN,Océano Atlántico Ltda.,Yvonne Moncada,Sales Agent,Ing. Gustavo Moncada 8585 Piso 20-A,Buenos Aires,NULL,1010,Argentina,...,1997-02-06 06:00:00,1997-01-14 00:00:00.000,1,29.83,Océano Atlántico Ltda.,Ing. Gustavo Moncada 8585 Piso 20-A,NULL,1010,Argentina,Buenos Aires
1,10448,RANCH,Rancho grande,Sergio Gutiérrez,Sales Representative,Av. del Libertador 900,Buenos Aires,NULL,1010,Argentina,...,1997-03-17 06:00:00,1997-02-24 00:00:00.000,2,38.82,Rancho grande,Av. del Libertador 900,NULL,1010,Argentina,Buenos Aires
2,10521,CACTU,Cactus Comidas para llevar,Patricio Simpson,Sales Agent,Cerrito 333,Buenos Aires,NULL,1010,Argentina,...,1997-05-27 05:00:00,1997-05-02 00:00:00.000,2,17.22,Cactus Comidas para llevar,Cerrito 333,NULL,1010,Argentina,Buenos Aires
3,10531,OCEAN,Océano Atlántico Ltda.,Yvonne Moncada,Sales Agent,Ing. Gustavo Moncada 8585 Piso 20-A,Buenos Aires,NULL,1010,Argentina,...,1997-06-05 05:00:00,1997-05-19 00:00:00.000,1,8.12,Océano Atlántico Ltda.,Ing. Gustavo Moncada 8585 Piso 20-A,NULL,1010,Argentina,Buenos Aires
4,10716,RANCH,Rancho grande,Sergio Gutiérrez,Sales Representative,Av. del Libertador 900,Buenos Aires,NULL,1010,Argentina,...,1997-11-21 06:00:00,1997-10-27 00:00:00.000,2,22.57,Rancho grande,Av. del Libertador 900,NULL,1010,Argentina,Buenos Aires
5,10782,CACTU,Cactus Comidas para llevar,Patricio Simpson,Sales Agent,Cerrito 333,Buenos Aires,NULL,1010,Argentina,...,1998-01-14 06:00:00,1997-12-22 00:00:00.000,3,1.10,Cactus Comidas para llevar,Cerrito 333,NULL,1010,Argentina,Buenos Aires
6,10819,CACTU,Cactus Comidas para llevar,Patricio Simpson,Sales Agent,Cerrito 333,Buenos Aires,NULL,1010,Argentina,...,1998-02-04 06:00:00,1998-01-16 00:00:00.000,3,19.76,Cactus Comidas para llevar,Cerrito 333,NULL,1010,Argentina,Buenos Aires
7,10828,RANCH,Rancho grande,Sergio Gutiérrez,Sales Representative,Av. del Libertador 900,Buenos Aires,NULL,1010,Argentina,...,1998-01-27 06:00:00,1998-02-04 00:00:00.000,1,90.85,Rancho grande,Av. del Libertador 900,NULL,1010,Argentina,Buenos Aires
8,10881,CACTU,Cactus Comidas para llevar,Patricio Simpson,Sales Agent,Cerrito 333,Buenos Aires,NULL,1010,Argentina,...,1998-03-11 06:00:00,1998-02-18 00:00:00.000,1,2.84,Cactus Comidas para llevar,Cerrito 333,NULL,1010,Argentina,Buenos Aires
9,10898,OCEAN,Océano Atlántico Ltda.,Yvonne Moncada,Sales Agent,Ing. Gustavo Moncada 8585 Piso 20-A,Buenos Aires,NULL,1010,Argentina,...,1998-03-20 06:00:00,1998-03-06 00:00:00.000,2,1.27,Océano Atlántico Ltda.,Ing. Gustavo Moncada 8585 Piso 20-A,NULL,1010,Argentina,Buenos Aires


The following cell runs a new **optimized SQL** query against the **partitioned data** that has been produced by the previous ETL SQL statement. The query uses `WHERE` predicates on the columns that have been used to partition the results in the ETL job. The query will physically only read the objects that match these predicate values.

In [20]:
optimized_sql='SELECT * FROM {} STORED AS PARQUET WHERE ShipCountry = "Austria" AND ShipCity="Graz" \
               INTO {} STORED AS PARQUET'.format(resultset_location, targeturl)
formatted_optimized_sql = sqlparse.format(optimized_sql, reindent=True, indent_tabs=True, keyword_case='upper')
result = highlight(formatted_optimized_sql, lexer, formatter)
print('\nRunning SQL against the previously produced hive style partitioned objects as input:\n')
print(result)

jobId = sqlClient.submit_sql(optimized_sql)
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
job_details = sqlClient.get_job(jobId)
if job_status == 'failed':
    print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))


Running SQL against the previously produced hive style partitioned objects as input:

SELECT *
FROM cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954 STORED AS PARQUET WHERE ShipCountry = "Austria" AND ShipCity="Graz"                INTO cos://us-south/sqltempregional/ STORED AS PARQUET

Job 6c394b3f-2339-4f9c-9433-b35bae399e13 terminated with status: completed


The following cell uses the `get_job()` method in order to show the job details of the just run optimized SQL that leverages hive style partitioning. Note the **bytes_read** value that is significantly lower than the **total_volume** value of the data in the queries data set. This does increase query performance and lower the query cost.

In [21]:
sqlClient.get_job(jobId)

{'job_id': '6c394b3f-2339-4f9c-9433-b35bae399e13',
 'status': 'completed',
 'statement': 'SELECT * FROM cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954 STORED AS PARQUET WHERE ShipCountry = "Austria" AND ShipCity="Graz"                INTO cos://us-south/sqltempregional/ STORED AS PARQUET',
 'plan_id': 'ead0f7f5-0c96-40c0-9aae-63c4846d8188',
 'submit_time': '2020-02-13T13:46:29.984Z',
 'resultset_location': 'cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/jobid=6c394b3f-2339-4f9c-9433-b35bae399e13',
 'rows_returned': 30,
 'rows_read': 30,
 'bytes_read': 6090,
 'resultset_format': 'parquet',
 'end_time': '2020-02-13T13:46:35.480Z',
 'user_id': 'torsten@de.ibm.com'}

### <a id="pagination"></a> 6. Paginated SQL Results
The next cell runs a simple join SQL. But this time `submit_sql()` is provided the optional **`pagesize`** parameter with a value of **`10`**. This results in multiple objects being written with each having 10 rows of the result in it. Internally this is achieved by using the SQL Query syntax clause of `PARTITIONED EVERY <num> ROWS`. This also means that your query cannot already contain another `PARTITIONED BY` clause.

In [22]:
pagination_sql='SELECT OrderID, c.CustomerID CustomerID, CompanyName, City, Region, PostalCode \
                FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, \
                     cos://us-geo/sql/customers.parquet STORED AS PARQUET c \
                WHERE c.CustomerID = o.CustomerID \
                INTO {}paginated_orders STORED AS PARQUET'.format(targeturl)
formatted_etl_sql = sqlparse.format(etl_sql, reindent=True, indent_tabs=True, keyword_case='upper')
result = highlight(formatted_etl_sql, lexer, formatter)
print('\nExample ETL Statement is:\n')
print(result)

jobId = sqlClient.submit_sql(pagination_sql, pagesize=10)
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
job_details = sqlClient.get_job(jobId)
if job_status == 'failed':
    print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))


Example ETL Statement is:

SELECT OrderID,
	c.CustomerID CustomerID,
	CompanyName,
	ContactName,
	ContactTitle,
	Address,
	City,
	Region,
	PostalCode,
	Country,
	Phone,
	Fax EmployeeID,
	OrderDate,
	RequiredDate,
	ShippedDate,
	ShipVia,
	Freight,
	ShipName,
	ShipAddress,
	ShipCity,
	ShipRegion,
	ShipPostalCode,
	ShipCountry
FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o,          cos://us-geo/sql/customers.parquet STORED AS PARQUET c          WHERE c.CustomerID = o.CustomerID          INTO cos://us-south/sqltempregional/customer_orders STORED AS PARQUET PARTITIONED BY (ShipCountry, ShipCity)

Job c22d0bc1-091a-440d-8de3-604b683b5855 terminated with status: completed


Let's check how many pages with each 10 rows have been written:

In [23]:
print("Number of pages written by job {}: {}".format(jobId, len(sqlClient.list_results(jobId))))

Number of pages written by job c22d0bc1-091a-440d-8de3-604b683b5855: 85


The following cell retrieves the first page of the result as a data frame. The desired page is specified as the optional parameter **`pagenumber`** to the `get_result()` method.

In [24]:
pagenumber=1
sqlClient.get_result(jobId, pagenumber=pagenumber).head(100)

,OrderID,CustomerID,CompanyName,City,Region,PostalCode
0,11011,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
1,10952,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
2,10835,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
3,10702,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
4,10692,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
5,10643,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
6,10926,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021
7,10759,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021
8,10625,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021
9,10308,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021


The following cell gets the next page. Run it multiple times in order to retrieve the subsequent pages, one page after the another.

In [25]:
pagenumber+=1
sqlClient.get_result(jobId, pagenumber).head(100)

,OrderID,CustomerID,CompanyName,City,Region,PostalCode
0,10856,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
1,10682,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
2,10677,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
3,10573,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
4,10535,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
5,10507,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
6,10365,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
7,11016,AROUT,Around the Horn,London,NULL,WA1 1DP
8,10953,AROUT,Around the Horn,London,NULL,WA1 1DP
9,10920,AROUT,Around the Horn,London,NULL,WA1 1DP


### <a id="results"></a> 7. Working with SQL result objects
This section shows some useful methods to work with result objects of SQLs. The following cell runs a simple SQL that produces a single partitioned parquet result:

In [28]:
sql="SELECT * FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET LIMIT 100 INTO {}first100orders.parquet JOBPREFIX NONE STORED AS PARQUET".format(targeturl)
jobId = sqlClient.submit_sql(sql)
sqlClient.wait_for_job(jobId)

'completed'

The next cell lists the result objects produced by the simple SQL job using the method `list_results()`:

In [30]:
sqlClient.list_results(jobId).head(100)

,ObjectURL,Size,Bucket,Object
0,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/first100orders.parquet,0,sqltempregional,first100orders.parquet
1,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/first100orders.parquet/_SUCCESS,0,sqltempregional,first100orders.parquet/_SUCCESS
2,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/first100orders.parquet/part-00000-fc1f11f6-f879-4e4a-840e-c1ebe70ae88b-c000-attempt_20200213135759_0021_m_000000_0.snappy.parquet,10271,sqltempregional,first100orders.parquet/part-00000-fc1f11f6-f879-4e4a-840e-c1ebe70ae88b-c000-attempt_20200213135759_0021_m_000000_0.snappy.parquet


As can be seen the SQL job always produces a logical hierarchy of object names. The path specified in the URI of the `INTO` clause above is used as a logical root "folder", which is nothing else than an empty object. It will normally include a suffix containing the unique jobId that produced this result. But because the queryabove used the optional clause `JOBPREFIX NONE` this suffix is ommitted, which means each time the same targer path will be used with that clause the previous list of objects in that logical folder will be overwritten.

#### Exact target result
If the result of the SQL job was not written in a partitioned way, you can now optionally restructure the objects to a single object with the exact name that was specified in the `INTO` clause of the SQL job. Thisis achieved with the method `rename_exact_result()`:

In [31]:
sqlClient.rename_exact_result(jobId)
sqlClient.list_results(jobId).head(100)

,ObjectURL,Size,Bucket,Object
0,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/first100orders.parquet,10271,sqltempregional,first100orders.parquet


As can be seen the result of the previous SQL is now exactly one object with the exact name from the path of the `INTO` clause.

### <a id="joblist"></a> 8. Working with your SQL Job Submission History
The following cell uses the `get_cos_summary()` method to get a statistical overview of the data in the **target location** in COS that has been used by the above queries in this notebook.

In [32]:
sqlClient.get_cos_summary(targeturl)

{'url': 'cos://us-south/sqltempregional/',
 'total_objects': 386,
 'total_volume': '94.4 MB',
 'oldest_object_timestamp': 'January 16, 2020, 14H:54M:24S',
 'newest_object_timestamp': 'February 13, 2020, 14H:06M:32S',
 'smallest_object_size': '0.0 B',
 'smallest_object': 'customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954',
 'largest_object_size': '23.3 MB',
 'largest_object': 'jobid=4a8ffab7-577f-46f3-950c-23680ba9e5fe/part-00000-624fcbf5-7a2e-4b82-b2d0-f179611a1261-c000-attempt_20200203083945_0021_m_000000_0.csv'}

The method `get_jobs()` provides you a dataframe with the **30 most recent SQL submissions** with all details. You can change the value `-1`for `display.max_colwidth` to a positive integer if you want to truncate the cell content to shrink the overall table display size.

In [33]:
pd.set_option('display.max_colwidth', -1)
job_history_df = sqlClient.get_jobs()
job_history_df.head(100)

,job_id,status,user_id,statement,resultset_location,submit_time,end_time,rows_read,rows_returned,bytes_read,error,error_message
0,0ddbc50b-1f42-4229-b2a6-1a94c1fef35d,completed,torsten@de.ibm.com,SELECT * FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET LIMIT 100 INTO cos://us-south/sqltempregional/first100orders.parquet JOBPREFIX NONE STORED AS PARQUET,cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/first100orders.parquet,2020-02-13T13:57:54.871Z,2020-02-13T13:58:00.809Z,830,100,30606,None,None
1,c22d0bc1-091a-440d-8de3-604b683b5855,completed,torsten@de.ibm.com,"SELECT OrderID, c.CustomerID CustomerID, CompanyName, City, Region, PostalCode FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE c.CustomerID = o.CustomerID INTO cos://us-south/sqltempregional/paginated_orders STORED AS PARQUET PARTITIONED EVERY 10 ROWS",cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/paginated_orders/jobid=c22d0bc1-091a-440d-8de3-604b683b5855,2020-02-13T13:46:37.438Z,2020-02-13T13:46:55.352Z,921,830,17111,None,None
2,6c394b3f-2339-4f9c-9433-b35bae399e13,completed,torsten@de.ibm.com,"SELECT * FROM cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954 STORED AS PARQUET WHERE ShipCountry = ""Austria"" AND ShipCity=""Graz"" INTO cos://us-south/sqltempregional/ STORED AS PARQUET",cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/jobid=6c394b3f-2339-4f9c-9433-b35bae399e13,2020-02-13T13:46:29.984Z,2020-02-13T13:46:35.480Z,30,30,6090,None,None
3,16cb6bcc-56e6-4c84-9dd2-6d9ee344e954,completed,torsten@de.ibm.com,"SELECT OrderID, c.CustomerID CustomerID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, ShipCity, ShipRegion, ShipPostalCode, ShipCountry FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE c.CustomerID = o.CustomerID INTO cos://us-south/sqltempregional/customer_orders STORED AS PARQUET PARTITIONED BY (ShipCountry, ShipCity)",cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/customer_orders/jobid=16cb6bcc-56e6-4c84-9dd2-6d9ee344e954,2020-02-13T13:45:35.754Z,2020-02-13T13:46:20.213Z,921,830,43058,None,None
4,2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName INTO cos://us-south/sqltempregional/ STORED AS CSV",cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/jobid=2a5e3d0f-bcc4-4294-8fb7-4ecdd8a3a855,2020-02-13T13:44:09.741Z,2020-02-13T13:44:43.700Z,1760,29,41499,None,None
5,12f04e72-04b0-4073-900c-7779573a1b4f,completed,torsten@de.ibm.com,"SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > ""1998-01-01"" ORDER BY c.CompanyName INTO cos://us-south/sqltempregional/ STORED AS CSV",cos://s3.us-south.cloud-object-storage.appdomain.cloud/sqltempregional/jobid=12f04e72-04b0-4073-900c-7779573a1b4f,2020-02-13T13:43:57.095Z,2020-02-13T13:44:08.075Z,1760,29,41499,None,None
6,d0ff50ee-c177-4080-b15d-629464cde3bb,completed,torsten@de.ibm.com,SELECT * FROM cos://us-geo/sql/employees.parquet STORED AS PARQUET LIMIT 10 INTO cos://

In [34]:
sqlClient.export_job_history(targeturl + "my_job_history/")

Exported 30 new jobs


In [46]:
pd.set_option('display.max_colwidth', 20)
sql = "SELECT * FROM {}my_job_history/ STORED AS PARQUET INTO {} STORED AS PARQUET".format(targeturl, targeturl)
sqlClient.run_sql(sql)

,job_id,status,user_id,statement,resultset_location,submit_time,end_time,rows_read,rows_returned,bytes_read,error,error_message,__index_level_0__
0,0ddbc50b-1f42-42...,completed,torsten@de.ibm.com,SELECT * FROM co...,cos://s3.us-sout...,2020-02-13T13:57...,2020-02-13T13:58...,830.0,100.0,30606.0,None,None,0
1,c22d0bc1-091a-44...,completed,torsten@de.ibm.com,"SELECT OrderID, ...",cos://s3.us-sout...,2020-02-13T13:46...,2020-02-13T13:46...,921.0,830.0,17111.0,None,None,1
2,6c394b3f-2339-4f...,completed,torsten@de.ibm.com,SELECT * FROM co...,cos://s3.us-sout...,2020-02-13T13:46...,2020-02-13T13:46...,30.0,30.0,6090.0,None,None,2
3,16cb6bcc-56e6-4c...,completed,torsten@de.ibm.com,"SELECT OrderID, ...",cos://s3.us-sout...,2020-02-13T13:45...,2020-02-13T13:46...,921.0,830.0,43058.0,None,None,3
4,2a5e3d0f-bcc4-42...,completed,torsten@de.ibm.com,SELECT o.OrderID...,cos://s3.us-sout...,2020-02-13T13:44...,2020-02-13T13:44...,1760.0,29.0,41499.0,None,None,4
5,12f04e72-04b0-40...,completed,torsten@de.ibm.com,SELECT o.OrderID...,cos://s3.us-sout...,2020-02-13T13:43...,2020-02-13T13:44...,1760.0,29.0,41499.0,None,None,5
6,d0ff50ee-c177-40...,completed,torsten@de.ibm.com,SELECT * FROM co...,cos://s3.us-sout...,2020-02-13T13:37...,2020-02-13T13:37...,9.0,9.0,8593.0,None,None,6
7,e05173ca-f38f-49...,completed,torsten@de.ibm.com,SELECT o.OrderID...,cos://s3.us-sout...,2020-02-13T13:31...,2020-02-13T13:31...,1760.0,29.0,41499.0,None,None,7
8,4f0811b4-e029-4e...,completed,torsten@de.ibm.com,SELECT o.OrderID...,cos://s3.us-sout...,2020-02-13T13:30...,2020-02-13T13:30...,1760.0,29.0,41499.0,None,None,8
9,5459f5d9-c9f4-46...,completed,torsten@de.ibm.com,SELECT * FROM co...,cos://s3.us-sout...,2020-02-13T12:57...,2020-02-13T12:57...,9.0,9.0,8593.0,None,None,9


### <a id="next"></a> 9. Next steps
In this notebook you learned how you can use the `ibmcloudsql` library in a Python notebook to submit SQL queries on data in IBM Cloud Object Storage and how you can interact with the query results. If you want to automate such an SQL query execution as part of your cloud solution, you can use the <a href="https://console.bluemix.net/openwhisk/" target="_blank">IBM Cloud Functions</a> framework. There is a dedicated SQL function available that lets you set up a cloud function to run SQL statements with IBM Cloud SQL Query. You can find the documentation for doing this <a href="https://hub.docker.com/r/ibmfunctions/sqlquery/" target="_blank" rel="noopener noreferrer">here</a>.

### <a id="authors"></a>Authors

**Torsten Steinbach**, Torsten is the lead architect for IBM Cloud Data Lake. Previously he has worked as IBM architect for a series of data management products and services, including DB2, PureData for Analytics and Db2 on Cloud.

<hr>
Copyright &copy; IBM Corp. 2020. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>